In [1]:
import pandas as pd
import os

In [8]:
file_list = []

for file in os.listdir("C:\\Users\\peter\\Desktop\\MSA_Fall19\\LinkedinScrape"):
    if file.endswith(".xlsx"):
        file_list.append(os.path.join("C:\\Users\\peter\\Desktop\\MSA_Fall19\\LinkedinScrape", file))

In [9]:
file_list

['C:\\Users\\peter\\Desktop\\MSA_Fall19\\LinkedinScrape\\pooja_saraiya.xlsx',
 'C:\\Users\\peter\\Desktop\\MSA_Fall19\\LinkedinScrape\\sydney_hill.xlsx',
 'C:\\Users\\peter\\Desktop\\MSA_Fall19\\LinkedinScrape\\yoan_simbafo.xlsx']

In [10]:
for fi in range(len(file_list)):
    df = pd.read_excel(file_list[fi], header=None)
    df = df.dropna(axis = 0, how ='all').reset_index(drop="True")
    print(df, '\n', "-------------------------------------------------------------------")

                0                                                  1
0            Link  https://www.linkedin.com/in/pooja-saraiya-86a2...
1            Name                                      Pooja Saraiya
2     Connections                                     14 connections
3           About                                                NaN
4      Experience                                                NaN
5             NaN                                             Target
6             NaN                    Customer Service Representative
7             NaN                               Mar 2017 â€“ Present
8       Education                                                NaN
9             NaN                           Georgia State University
10            NaN                                      2016 â€“ 2020
11            NaN                                  Bachelor's degree
12            NaN                       Computer Information Systems
13  Organizations                 

In [12]:
result_df = pd.DataFrame(columns = df[0].dropna())
result_df

,Link,Name,Connections,About,Experience,Education,Organizations


In [13]:
for fi in range(len(file_list)):
    df = pd.read_excel(file_list[fi], header=None)
    df = df.dropna(axis = 0, how ='all').reset_index(drop="True")

    idx = df.index[df[0].notnull() == True].tolist()

    rowe = [ ]

    for i in range(len(idx[0:3])):
        tite = df[0][idx[i]]
        vale = df[1][idx[i]]
        tupe = (tite, vale)
        #print(tupe)

        rowe.append(vale)



    for i in range(len(idx[3:])):
        tite = df[0][idx[3:][i]]

        try:
            vale = '. '.join([x for x in df[1][idx[3:][i]+1:idx[3:][i+1]]])
        except:
            vale = "..."

        tupe = (tite, vale)
        #print(tupe, "\n", "----------------------------------------------")

        rowe.append(vale)

    rowe_sr = pd.Series(rowe, index = result_df.columns)
    result_df = result_df.append(rowe_sr, ignore_index=True)

    result_df

In [14]:
result_df

,Link,Name,Connections,About,Experience,Education,Organizations
0,https://www.linkedin.com/in/pooja-saraiya-86a2...,Pooja Saraiya,14 connections,,Target. Customer Service Representative. Mar 2...,Georgia State University. 2016 â€“ 2020. Bache...,...
1,https://www.linkedin.com/in/sydney-hill-922614162,Sydney Hill,106 connections,,Georgia State University College of Law. Assis...,Georgia State University College of Law. 2014 ...,...
2,https://www.linkedin.com/in/yoan-simbafo-8830b298,Yoan Simbafo,193 connections,Upcoming Loan Servicing Coordinator at Groundf...,GROUNDFLOOR Finance. Loan Servicing Coordinato...,Georgia State University - J. Mack Robinson Co...,...


In [15]:
#-------------Generate Some NLP features----------------------------

In [17]:
import spacy

In [18]:
nlp = spacy.load("en_core_web_sm")
# --> python -m spacy download en_core_web_sm --> install spacy models

In [20]:
result_df['About_len'] = result_df['About'].apply(lambda x: len(x))
result_df['Exp_len'] = result_df['Experience'].apply(lambda x: len(x))
result_df['Edu_len'] = result_df['Education'].apply(lambda x: len(x))

In [22]:
result_df['About_lines'] = result_df['About'].apply(lambda x: x.count('.'))
result_df['Exp_lines'] = result_df['Experience'].apply(lambda x: x.count('.'))
result_df['Edu_lines'] = result_df['Education'].apply(lambda x: x.count('.'))

In [58]:
result_df

,Link,Name,Connections,About,Experience,Education,Organizations,About_len,Exp_len,Edu_len,About_lines,Exp_lines,Edu_lines
0,https://www.linkedin.com/in/pooja-saraiya-86a2...,Pooja Saraiya,14 connections,,Target. Customer Service Representative. Mar 2...,Georgia State University. 2016 â€“ 2020. Bache...,...,0,61,88,0,2,3
1,https://www.linkedin.com/in/sydney-hill-922614162,Sydney Hill,106 connections,,Georgia State University College of Law. Assis...,Georgia State University College of Law. 2014 ...,...,0,361,153,0,13,6
2,https://www.linkedin.com/in/yoan-simbafo-8830b298,Yoan Simbafo,193 connections,Upcoming Loan Servicing Coordinator at Groundf...,GROUNDFLOOR Finance. Loan Servicing Coordinato...,Georgia State University - J. Mack Robinson Co...,...,58,1859,226,0,30,8


In [78]:
df = pd.DataFrame(columns=['Avg_Wrd_Len','Orgs','Org_Count','Ent_Count'], index=range(len(result_df)))

for index, row in result_df.iterrows():
    
    doc = nlp(result_df['Experience'][index])
    words = [t.text for t in doc]
    average = sum(len(word) for word in words) / len(words)
    
    orgs = [ ]

    for ent in doc.ents:
        if ent.label_ == "ORG":
            orgs.append(ent.text)

    org_count = len(orgs)
    ents_count = len([e.text for e in doc.ents])
    
    df['Avg_Wrd_Len'][index] = average
    df['Orgs'][index] = orgs
    df['Org_Count'][index] = org_count
    df['Ent_Count'][index] = ents_count

In [79]:
df

,Avg_Wrd_Len,Orgs,Org_Count,Ent_Count
0,4.90909,[],0,0
1,4.01316,"[Georgia State University College of Law, Geor...",5,14
2,4.79331,"[Loan Servicing Coordinator, Berkadia, HUD, As...",7,23


In [ ]:
#avg word lenght
#count of entities
#

In [64]:
index = 1

doc = nlp(result_df['Experience'][index])
words = [t.text for t in doc]
average = sum(len(word) for word in words) / len(words)


orgs = [ ]

for ent in doc.ents:
    if ent.label_ == "ORG":
        orgs.append(ent.text)
        
org_count = len(orgs)
ents_count = len([e.text for e in doc.ents])

df = pd.DataFrame(columns=['Avg_Wrd_Len','Org_Count','Ent_Count'], index=range(len(result_df)))

# df['Avg_Wrd_Len'][index] = average
# df['Orgs'][index] = orgs
# df['Org_Count'][index] = org_count
# df['Ent_Count'][index] = ents_count

In [62]:
len(result_df)

3

In [65]:
df

,Avg_Wrd_Len,Org_Count,Ent_Count
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN


In [75]:
def text_features(df, column):
    for index, row in result_df.iterrows:
    
        doc = nlp(df[column][index])
        words = [t.text for t in doc]
        average = sum(len(word) for word in words) / len(words)


        orgs = [ ]

        for ent in doc.ents:
            if ent.label_ == "ORG":
                orgs.append(ent.text)

        org_count = len(orgs)
        ents_count = len([e.text for e in doc.ents])

        df = pd.DataFrame(columns=['Avg_Wrd_Len','Org_Count','Ent_Count'], index=range(len(result_df)))

        df['Avg_Wrd_Len'][index] = average
        df['Org_Count'][index] = org_count
        df['Ent_Count'][index] = ents_count
    
    return df
    

In [80]:
def text_features(df, column):

    df = pd.DataFrame(columns=['Avg_Wrd_Len','Orgs','Org_Count','Ent_Count'], index=range(len(result_df)))

    for index, row in result_df.iterrows():

        doc = nlp(result_df['Experience'][index])
        words = [t.text for t in doc]
        average = sum(len(word) for word in words) / len(words)

        orgs = [ ]

        for ent in doc.ents:
            if ent.label_ == "ORG":
                orgs.append(ent.text)

        org_count = len(orgs)
        ents_count = len([e.text for e in doc.ents])

        df['Avg_Wrd_Len'][index] = average
        df['Orgs'][index] = orgs
        df['Org_Count'][index] = org_count
        df['Ent_Count'][index] = ents_count
        
    return df

In [83]:
exp_df = text_features(result_df, 'Experience')

exp_df  # <--Can append this to the main dataframe if you want to

In [85]:
edu_df = text_features(result_df, 'Eduation')

edu_df  # <--Can append this to the main dataframe if you want to

,Avg_Wrd_Len,Orgs,Org_Count,Ent_Count
0,4.90909,[],0,0
1,4.01316,"[Georgia State University College of Law, Geor...",5,14
2,4.79331,"[Loan Servicing Coordinator, Berkadia, HUD, As...",7,23
